# Problem description from Frank
One task that you can already start working on (since this was part of our first meeting)
is a numerical example of one complete forward backward learning cycle through a very simple
classification network consisting of a feature extraction/learning part (CNN with conv. and
pooling layers) and a classification part (FCNN with at least one fully-connected/dens layer)

I choose to break this down into the following subtasks


## 1. Create simple FCNN module
Inspiration from micrograd can be taken.

The FCNN module can be built as follows:
1. Class for a single neuron, with weights and a bias. The dunder-call for this should return the activation of the weighted sum of inputs. The activation function should be specifiable from a discrete set of types.
2. Class for a layer. A layer should have a specifiable amount of neurons, and the dunder-call should be a call to each neuron, all using the same input.


## 2. Create CNN w. conv module
## 3. Connect CNN->FCNN
## 4. Numerical example of forward pass
## 5. Numerical example of backward pass


In [1]:
# Imports here

import numpy as np

In [47]:

from dataclasses import dataclass
from turtle import backward
from typing import Callable

from __future__ import annotations

@dataclass
class Float():
    """A wrapper class for numbers in the network that adds additional
    functionality like storing the gradient to allow for easy backward
    propagation
    """
    value: float
    children: set = ()
    grad: float = 0
    backward: Callable[[], None] = None


    # def __post_init__(self):
    #     self.backward = lambda: None

    def __cast_to_self(self, other):
        return other if isinstance(other, Float) else Float(other)

    def __add__(self, other) -> Float:
        other = self.__cast_to_self(other)
        out = Float(self.value + other.value, (self, other))

        def backward() -> None:
            self.grad += out.grad
            other.grad += out.grad

        out.backward = backward
        
        return out

    def __mul__(self, other) -> Float:
        other = self.__cast_to_self(other)
        out = Float(self.value * other.value, (self, other))

        def backward() -> None:
            self.grad += other.value * out.grad
            other.grad += self.value * out.grad
        
        out.backward = backward

        return out

    def __pow__(self, other) -> Float:
        other = self.__cast_to_self(other)
        out = Float(self.value**other, (self,))

        def backward() -> None:
            self.grad += (other * self.value**(other - 1)) * out.grad

        out.backward = backward

        return out

    def __neg__(self) -> Float:
        return self * -1


    def __radd__(self, other) -> Float:
        return self + other

    def __rmul__(self, other) -> Float:
        return self * other

    def __repr__(self) -> str:
        return f"Float(data={self.value}, grad={self.grad})"


    # Below is a collection of activation functions 
    def relu(self) -> Float:
        value = max(0, self.value)
        out = Float(value, (self,))

        def backward() -> None:
            self.grad += (value > 0) * out.grad

        out.backward = backward()

        return out
        
    def tanh(self):
        raise NotImplementedError


    def backward_pass(self):
        """The main algorithm for a backwards pass. Works by topologically
        sorting all connected nodes then iteratively and reversely calling
        backwards on the sorted list.
        """
        nodes_sorted = []
        nodes_visited = []

        def topological_sort(node) -> None:
            if node not in nodes_visited:
                nodes_visited.append(node)

                for child in node.children:
                    #print(type(child))
                    topological_sort(child)
                nodes_sorted.append(node)

        topological_sort(self)

        self.grad = 1 # Need to set this to non-zero, else all grads will be 0
        #print(nodes_sorted)
        for node in reversed(nodes_sorted):
            print(node)
            print(type(node))
            node.backward()





# Test
# a = Float(10, 100)
# b  = Float(10, 1)
# print(a.value)
# print(a.grad)

x = Float(-4.0)
z = 2 * x + 2 + x
q = z.relu() + z * x
h = (z * z).relu()
y = h + q + q * x
y.backward_pass()


Float(data=-20.0, grad=1)
<class '__main__.Float'>
Float(data=-160.0, grad=1)
<class '__main__.Float'>
Float(data=140.0, grad=1)
<class '__main__.Float'>
Float(data=40.0, grad=-3.0)
<class '__main__.Float'>
Float(data=40.0, grad=-3.0)
<class '__main__.Float'>
Float(data=0, grad=-3.0)
<class '__main__.Float'>


TypeError: 'NoneType' object is not callable

In [ ]:
class Base():
    
    def reset(self):
        for p in self.p():
            p.grad = 0

    def p(self):
        return []

In [2]:

class Neuron():
    
    def __init__(self, n_inputs, f=None):
        """Create a neuron with n inputs.
        
        Optionally specify an activation function.
        If no activation function is set, a default one
        will be set. Right now that is f(x) = x
        """
        self.w = np.random.uniform(-1, 1, n_inputs)
        self.b = np.random.uniform()
        
        if f is None:
            self.f = lambda x : x # Linear
            #self.f = lambda x : max(x, 0) #ReLU
        else:
            self.f = f
    
    def z(self, x):
        """Calculate the pre-activation neuron output"""
        return x@self.w.T + self.b
    
    def a(self, x):
        """Calculate the post-activation neuron output"""
        return self.f(self.z(x))

    def p(self):
        """Return the weights of the neuron"""
        return [self.b] + self.w

    
# Test
#N = 3
#n = Neuron(N)
#x = np.random.rand(N)
#print(n.a(x))


In [3]:
class Layer():
    
    def __init__(self, n_neurons, n_inputs):
        """Create a layer of n_neurons
        
        The amount of inputs to the neurons will also need to be specified.
        """
        self.neurons = [Neuron(n_inputs) for _ in range(n_neurons)]
        
    def out(self, x):
        """Return the vector of outputs of the layer"""
        return [n.out(x) for n in self.neurons]

    def p(self):
        """Return the parameters for all the neurons in the layer"""
        return [p for n in self.neurons for p in n.p()]
        
    
        
    

In [4]:
class FCNN():
    

    def __init__(self, n_inputs, n_outputs):
        """Create a fully-connected network of n layers
        """
        sizes = [n_inputs] + n_outputs
        self.layers = [Layer(sizes[i], sizes[i+1]) for i in range(len(n_outputs))]
        
        
    def out(self, x):
        for l in self.layers:
            x = l(x)
        return x

    def p(self):
        [p for l in self.layers for p in l.p()]
